In [ ]:
timm_path = "../input/timm-pytorch-image-models/pytorch-image-models-master"
import sys
sys.path.append(timm_path)
import timm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import os
from tqdm.notebook import tqdm
import pydicom
import matplotlib.pyplot as plt
import glob
import sklearn
import random

import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader
from torch import optim

import warnings
warnings.filterwarnings('ignore')
np.seterr(divide='ignore', invalid='ignore')

import nibabel as nib
import SimpleITK as sitk
import shutil
from PIL import Image
from IPython.display import clear_output
import glob
import time
from shutil import copyfile
from statistics import mean

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
if not os.path.exists('/kaggle/working/yolov5'):
    shutil.copytree('/kaggle/input/yolov5-official-v31-dataset/yolov5', '/kaggle/working/yolov5')

In [ ]:
train_path = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/train'
test_path = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/test'
label_dir = '/kaggle/working/runs/detect/exp/labels'
image_predict_dir = '/kaggle/working/predict'
prediction_results_dir = '/kaggle/working/runs/detect/exp/'
test_dir = '/kaggle/working/test'

series = 'T1wCE'
plane = 'axial'

predictions = []
preds = []
study_count = 0
studies_without_preds = []
ref_plane = ''

reader = sitk.ImageSeriesReader()
reader.LoadPrivateTagsOn()

In [ ]:
def detect():    
    !python yolov5/detect.py --source '{image_predict_dir}' --weights '../input/rsna-winning-models/brain_tumor_axial_t1wce.pt' --img 512 --exist-ok --save-txt


In [ ]:
def make_dirs(study, series):
    if not os.path.exists(test_dir):
        os.mkdir(test_dir)
    if not os.path.exists(f'{test_dir}/{study}'):
        os.mkdir(f'{test_dir}/{study}')
    if not os.path.exists(f'{test_dir}/{study}/{series}'):
        os.mkdir(f'{test_dir}/{study}/{series}')

    if not os.path.exists(image_predict_dir):
        os.mkdir(image_predict_dir)
        
def get_image_plane(loc):
    row_x = round(loc[0])
    row_y = round(loc[1])
    row_z = round(loc[2])
    col_x = round(loc[3])
    col_y = round(loc[4])
    col_z = round(loc[5])
    if row_x == 1 and row_y == 0 and col_x == 0 and col_y == 0:
        return "Coronal"
    if row_x == 0 and row_y == 0 and col_x == 1 and col_y == 0:
        return "Sagittal"
    if row_x == 1 and row_y == 0 and col_x == 0 and col_y == 1:
        return "Axial"
    return "Unknown"

# This function does all the voxel resampling and orienting
def resample(image, ref_image):
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(ref_image)
    resampler.SetInterpolator(sitk.sitkLinear) 
    resampler.SetTransform(sitk.AffineTransform(image.GetDimension()))
    resampler.SetOutputSpacing(ref_image.GetSpacing())
    resampler.SetSize(ref_image.GetSize())
    resampler.SetOutputDirection(ref_image.GetDirection())
    resampler.SetOutputOrigin(ref_image.GetOrigin())
    resampler.SetDefaultPixelValue(image.GetPixelIDValue())
    resamped_image = resampler.Execute(image)
    return resamped_image

def normalize_pixels(pixels):
    pixels = pixels - np.min(pixels)
    pixels = pixels / np.max(pixels)
    pixels = (pixels * 255).astype(np.uint8)
    return pixels
def detect_on_image_set(study, series):
    
    global plane
    images = []
    list = os.listdir(f'{test_dir}/{study}/{series}')

    if len(list) > 5:
        df = pd.DataFrame({'filename':list})
        df['filename'] = pd.to_numeric(df['filename'].astype(str).str[:-4])
        df = df.sort_values(by=['filename'])

        center_image = int(df.shape[0] / 2)
        if plane == 'sagittal':
            images.append(df.iloc[int(center_image / 7 * 2)]['filename'])
            
        images.append(df.iloc[int(center_image / 7 * 3)]['filename'])
        images.append(df.iloc[int(center_image / 7 * 4)]['filename'])
        images.append(df.iloc[center_image]['filename'])
        images.append(df.iloc[center_image + int(center_image / 7)]['filename'])
        images.append(df.iloc[center_image + int(center_image / 7 * 2)]['filename'])
        images.append(df.iloc[center_image + int(center_image / 7 * 3)]['filename'])

        if plane == 'axial':
            images.append(df.iloc[center_image + int(center_image / 7 * 4)]['filename'])
            
        for img in images:
            copyfile(f'{test_dir}/{study}/{series}/{img}.jpg',f'/{image_predict_dir}/{img}.jpg')
            
        clear_output()
        print("PREDICTING -> ", study_count)
        detect()

In [ ]:
def cleanup(study):

    filelist = [ f for f in os.listdir(label_dir) if f.endswith(".txt") ]
    for f in filelist:
        os.remove(os.path.join(label_dir, f))

    filelist = [ f for f in os.listdir(prediction_results_dir) if f.endswith(".jpg") ]
    for f in filelist:
        copyfile(f'{prediction_results_dir}/{f}',f'/{test_dir}/{study}/{series}/pred_{f}')
        os.remove(os.path.join(prediction_results_dir, f))
    
    for f in os.listdir(image_predict_dir):
        os.remove(os.path.join(image_predict_dir, f))

In [ ]:
def test_study(study):
    
    global study_count
    counter = 0

    make_dirs(study, series)
    target_array = get_target_array(study)
    for i in range(0,target_array.shape[0]):
        if counter == 0:
            if len(target_array) > 0:
                pixels = normalize_pixels(target_array[i,:,:])
                w = target_array.shape[0] - 50
                h = target_array.shape[1] - 50
                pix_mean = np.mean(target_array[i,50:w,50:h])
                if pix_mean > 40.0:
                    filename = f'{test_dir}/{study}/{series}/{i}.jpg'
                    cv2.imwrite(filename, pixels)             
        counter += 1
        if counter == 3:
            counter = 0

    detect_on_image_set(study, series)

    filelist = [ f for f in os.listdir(label_dir) if f.endswith(".txt") ]
    
    if len(filelist) < 1:
        studies_without_preds.append(study)

    for f in filelist:
        file = f.split('.')
        p =  f'{test_dir}/{study}/{series}/{file[0]}.jpg'
        preds.append(p)     

    cleanup(study)
    study_count += 1

In [ ]:
def get_target_array(study):

    target_files = reader.GetGDCMSeriesFileNames(f'{test_path}/{study}/{series}')
    reader.SetFileNames(target_files)
    target_set = reader.Execute()

    target_plane = get_image_plane(target_set.GetDirection())

    # Only resample images that aren't already in the reference plane, otherwise just export as-is.
    if ref_plane != target_plane:
        target_new = resample(target_set, ref_set)
    else:
        target_new = target_set

    return sitk.GetArrayFromImage(target_new)

    target_files = reader.GetGDCMSeriesFileNames(f'{test_path}/{study}/{series}')
    reader.SetFileNames(target_files)
    target_set = reader.Execute()
    target_plane = get_image_plane(target_set.GetDirection())
    if ref_plane != target_plane:
        target_new = resample(target_set, ref_array)
    else:
        target_new = target_set
    return sitk.GetArrayFromImage(target_new)

In [ ]:
if plane == 'axial':
    reference_series = '00000/T1w'
if plane == 'sagittal':
    reference_series = '00000/T2w'

ref_files = reader.GetGDCMSeriesFileNames(f'{train_path}/{reference_series}')
reader.SetFileNames(ref_files)
ref_set = reader.Execute()

ref_plane = get_image_plane(ref_set.GetDirection())
ref_array = sitk.GetArrayFromImage(ref_set)
    

print("Reference Image Plane: " + ref_plane)

In [ ]:
start_time = time.time()

test_studies = [ f for f in os.listdir(test_path) ]
len(test_studies)

In [ ]:
start_time = time.time()

test_studies = [ f for f in os.listdir(test_path) ]

if( len(test_studies) > 87):
    for study in test_studies:
        test_study(study)
else:
    test_studies = test_studies[:3]
    for study in test_studies:
        test_study(study)
     
clear_output()
end_time = time.time()

total_time = round((end_time - start_time) / 60, 2)
print("TOTAL TIME: ", total_time, " min")

In [ ]:
preds = sorted(preds)

In [ ]:
image_size = 384
aug = A.Compose(
    [  A.Resize(image_size,image_size,p=1.0),      
        A.CenterCrop(height = 270 ,width = 270,p=1.0), 
         A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
     A.Transpose(p=0.5),   
        A.RandomContrast(limit=0.2, p=0.5),
        A.Resize(image_size,image_size,p=1.0),      
        ToTensorV2(p=1.0)
    ]
)

In [ ]:
class Brain(Dataset):
    def __init__(self , path , augs=aug):
        self.p = path
        self.augs = augs
        
    def __len__(self):
        return(len(self.p))
    
    def __getitem__(self , idx):
        img_src = self.p[idx]
        image = cv2.imread(img_src)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.uint8)

        if (self.augs):
            transformed = self.augs(image=image)
            image = transformed['image']
            image = (image)/255.
            image = torch.tensor(image,dtype = torch.float32) 
       
    
        return image

In [ ]:
data = Brain(preds)
load = DataLoader(data,batch_size = 1 , shuffle = True)
img = next(iter(load))
plt.imshow(img.squeeze(0).permute(1,2,0))

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = timm.create_model('efficientnet_b3', pretrained=False,  num_classes=0, global_pool='')
        self.pool = nn.AdaptiveAvgPool2d(1)
        #n_in = self.net.classifier.in_features
        self.fc = nn.Linear(1536 , 128)
        self.fc2 =  nn.Linear(128 , 1)
        self.dp = nn.Dropout(p=0.25)
    
    def forward(self, x):
        x = self.net(x)
        x = self.pool(x)
        x = x.view(x.shape[0],-1)
        x = F.relu(self.fc(x))
        x = self.dp(x)
        x= self.fc2(x)
        
        return x    

In [ ]:
x = "../input/rsna-winning-models/2D_Fold model with val_auc 0.6829104010025063 and train auc 0.7496031746031746.pth"
mpath = [x]

In [ ]:
test_dataset = Brain(preds)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, shuffle=False,  num_workers=4)

In [ ]:
def inference_func(test_loader , Model):
    Model.eval()
    bar = tqdm(test_loader)

    PREDS = []
    
    with torch.no_grad():
        for batch_idx, images in enumerate(bar):
            x = images.to(device)
            output = Model(x)
            PREDS += [F.sigmoid(output).detach().cpu()]
        PREDS = torch.cat(PREDS).cpu().numpy()  
    return PREDS

In [ ]:
predx1 = []
predx2 = []
predx3 = []
predx4 = []
for m in mpath:
    mod = Model()
    mod = mod.to(device)
    mod.load_state_dict(torch.load(m))
    predx1 += [inference_func(test_loader , mod)]
for m in mpath:
    mod = Model()
    mod = mod.to(device)
    mod.load_state_dict(torch.load(m))
    predx2 += [inference_func(test_loader , mod)]
for m in mpath:
    mod = Model()
    mod = mod.to(device)
    mod.load_state_dict(torch.load(m))
    predx3 += [inference_func(test_loader , mod)]
for m in mpath:
    mod = Model()
    mod = mod.to(device)
    mod.load_state_dict(torch.load(m))
    predx4 += [inference_func(test_loader , mod)]

In [ ]:
y_pa = torch.from_numpy(np.mean(predx1, axis=0)).numpy()
cata = np.concatenate((np.expand_dims(np.array(preds), axis=1), y_pa), axis = 1)

y_pb = torch.from_numpy(np.mean(predx2, axis=0)).numpy()
catb = np.concatenate((np.expand_dims(np.array(preds), axis=1), y_pb), axis = 1)

y_pc = torch.from_numpy(np.mean(predx3, axis=0)).numpy()
catc = np.concatenate((np.expand_dims(np.array(preds), axis=1), y_pc), axis = 1)

y_pd = torch.from_numpy(np.mean(predx4, axis=0)).numpy()
catd = np.concatenate((np.expand_dims(np.array(preds), axis=1), y_pd), axis = 1)

In [ ]:
sub=pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/sample_submission.csv')
sub["imfolder"] = ['{0:05d}'.format(s) for s in sub["BraTS21ID"]]

In [ ]:
sub['t1'] = 0.5
sub['t2'] = 0.5
sub['t3'] = 0.5
sub['t4'] = 0.5
sub['t5'] = 0.5
sub['t6'] = 0.5
sub['t7'] = 0.5
sub['t8'] = 0.5

In [ ]:
for i in sub["imfolder"]:
    r = 0
    for j in range(len(cata)):
        if(r == 0):
            if(i == cata[j][0][21:26]):
                sub.loc[sub['imfolder'] == i,'t1'] = np.float(cata[j][1])
                r = r+1
for i in sub["imfolder"]:
    for j in range(len(cata)):
        if(r == 0):
            if(i == cata[j][0][21:26]):
                sub.loc[sub['imfolder'] == i,'t2'] = np.float(cata[j][1])

In [ ]:
for i in sub["imfolder"]:
    r = 0
    for j in range(len(catb)):
        if(r == 0):
            if(i == catb[j][0][21:26]):
                sub.loc[sub['imfolder'] == i,'t3'] = np.float(catb[j][1])
                r = r+1
for i in sub["imfolder"]:
    for j in range(len(catb)):
        if(r == 0):
            if(i == catb[j][0][21:26]):
                sub.loc[sub['imfolder'] == i,'t4'] = np.float(catb[j][1])

In [ ]:
for i in sub["imfolder"]:
    r = 0
    for j in range(len(catc)):
        if(r == 0):
            if(i == catc[j][0][21:26]):
                sub.loc[sub['imfolder'] == i,'t5'] = np.float(catc[j][1])
                r = r+1
for i in sub["imfolder"]:
    for j in range(len(catc)):
        if(r == 0):
            if(i == catc[j][0][21:26]):
                sub.loc[sub['imfolder'] == i,'t6'] = np.float(catc[j][1])

In [ ]:
for i in sub["imfolder"]:
    r = 0
    for j in range(len(catd)):
        if(r == 0):
            if(i == catd[j][0][21:26]):
                sub.loc[sub['imfolder'] == i,'t7'] = np.float(catd[j][1])
                r = r+1
for i in sub["imfolder"]:
    for j in range(len(catc)):
        if(r == 0):
            if(i == catd[j][0][21:26]):
                sub.loc[sub['imfolder'] == i,'t8'] = np.float(catd[j][1])

In [ ]:
sub['MGMT_value'] = (sub['t1']**2 + sub['t2']**2 + sub['t3']**2 + sub['t4']**2 + sub['t5']**2 + sub['t6']**2 + sub['t7']**2 + sub['t8']**2)/8

In [ ]:
pathh = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/test'
tes = [os.path.join(pathh,x) for x in studies_without_preds]

In [ ]:
from pydicom.pixel_data_handlers.util import apply_voi_lut
def load_dicom(path , image_size,voi_lut=True, fix_monochrome=True  ):
    dicom = pydicom.read_file(path)
    # VOI LUT (if available by DICOM device) is used to
    # transform raw DICOM data to "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    
    data = cv2.resize(data, (image_size, image_size))
    return data

def load_3d(path , image_size  , depth =48):
    flair = sorted(glob.glob(f"{path}/FLAIR/*.dcm"))
    t1w = sorted(glob.glob(f"{path}/T1w/*.dcm"))
    t1wce = sorted(glob.glob(f"{path}/T1wCE/*.dcm"))
    t2w = sorted(glob.glob(f"{path}/T2w/*.dcm"))
    
    depth_per_source = depth // 4
    s = 0
    flair_img = np.array([load_dicom(a , image_size) for a in flair[s::depth_per_source]]).T

    if flair_img.shape[-1] < depth_per_source:
        n_zero = depth_per_source - flair_img.shape[-1]
        flair_img = np.concatenate((flair_img, np.zeros((image_size, image_size, n_zero))), axis = -1)
    elif (flair_img.shape[-1] > depth_per_source):
        n = flair_img.shape[-1] - depth_per_source
        flair_img = flair_img[:,:,:-n]
        
    t1w_img = np.array([load_dicom(a , image_size) for a in t1w[s::depth_per_source]]).T

    if t1w_img.shape[-1] < depth_per_source:
        n_zero = depth_per_source - t1w_img.shape[-1]
        t1w_img = np.concatenate((t1w_img, np.zeros((image_size, image_size, n_zero))), axis = -1)
    elif (t1w_img.shape[-1] > depth_per_source):
        n = t1w_img.shape[-1] - depth_per_source
        t1w_img = t1w_img[:,:,:-n]
        
    t1wce_img = np.array([load_dicom(a , image_size) for a in t1wce[s::depth_per_source]]).T

    if t1wce_img.shape[-1] < depth_per_source:
        n_zero = depth_per_source - t1wce_img.shape[-1]
        t1wce_img = np.concatenate((t1wce_img, np.zeros((image_size, image_size, n_zero))), axis = -1)
    elif (t1wce_img.shape[-1] > depth_per_source):
        n = t1wce_img.shape[-1] - depth_per_source
        t1wce_img = t1wce_img[:,:,:-n]
    
    t2w_img = np.array([load_dicom(a , image_size) for a in  t2w[s::depth_per_source]]).T

    if t2w_img.shape[-1] < depth_per_source:
        n_zero = depth_per_source - t2w_img.shape[-1]
        t2w_img = np.concatenate((t2w_img, np.zeros((image_size, image_size, n_zero))), axis = -1)
    elif (t2w_img.shape[-1] > depth_per_source):
        n = t2w_img.shape[-1] - depth_per_source
        t2w_img = t2w_img[:,:,:-n]
    
   
    
    image_3d =  np.concatenate((flair_img, t1w_img, t1wce_img, t2w_img), axis = -1) #shpe= height x width x depth
    image_3d = torch.tensor(image_3d , dtype = torch.float32 ).permute(2 , 1 ,0)
    
    return image_3d

In [ ]:
class BrainX(Dataset):
    def __init__(self , pre , image_size = 256 , depth = 48):
        self.pre = pre
        self.img_size =image_size
        self.d = depth
    
    def __len__(self):
        return(len(self.pre))
    
    def __getitem__(self , idx):
        img_path = self.pre[idx]
        img_3d = load_3d(img_path , self.img_size , self.d)
        img_3d = img_3d/255.
        
        img_3d = img_3d - 0.5
        img_3d = (img_3d)/(0.5)
        
        img_3d = img_3d.unsqueeze(0) # channel_length , deth , width , height
       
        return img_3d 

In [ ]:
model=timm.create_model('efficientnet_b1', pretrained=False) # set pretrained=True to use the pretrained weights
num_features = model.classifier.in_features
model.classifier = nn.Linear(num_features, 1)
class Model_3d_2_2d(nn.Module):
    def __init__(self, model , input_channels = 1):
        super().__init__()
        self.model = model
        self.cnn3d = nn.Conv3d( input_channels , 1 ,kernel_size = (8,3,3) , stride=(2, 1, 1))
        self.pool1 = nn.AdaptiveMaxPool3d((8 , 224,224))
        self.cnn2d = nn.Conv2d(8, 3 , 3 , stride = 1 , padding = 1 , bias = False)
        self.pool2 = nn.AdaptiveMaxPool2d((224))
        self.norm = nn.BatchNorm2d(3)
 
    def forward(self , x):
        x = F.relu(self.cnn3d(x))
        x = x.squeeze(1)
        x = self.pool1(x) 
        
        x = self.cnn2d(x)
        x = self.norm(x)
        x = F.relu(x)
        #print(x.shape)
        x = self.pool2(x)
        x = self.model(x)
        
        return x

In [ ]:
x = "../input/rsna-winning-models/3D_Fold 1 model with val_acc 0.6936425264550264 and train auc 0.6745247688135619.pth"
#y = "../input/best-cv/Fold 1 model with val_acc 0.7113839285714286 and train auc 0.8488540780207449.pth"
mpath = [x]

In [ ]:
test_dataset = BrainX(tes,  depth = 48 )
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, shuffle=False,  num_workers=4)

In [ ]:
def inference_func(test_loader , Model):
    Model.eval()
    bar = tqdm(test_loader)

    PREDS = []
    
    with torch.no_grad():
        for batch_idx, images in enumerate(bar):
            x = images.to(device)
            output = Model(x)
            PREDS += [F.sigmoid(output).detach().cpu()]
        PREDS = torch.cat(PREDS).cpu().numpy()  
    return PREDS

In [ ]:
preds = []
for m in mpath:
    mod = Model_3d_2_2d(model)
    mod = mod.to(device)
    mod.load_state_dict(torch.load(m))
    preds += [inference_func(test_loader , mod)]

In [ ]:
y_p = torch.from_numpy(np.mean(preds, axis=0)).numpy()

In [ ]:
cat2 = np.concatenate((np.expand_dims(np.array(studies_without_preds), axis=1), y_p), axis = 1)

In [ ]:
for i in sub["imfolder"]:
    r = 0
    for j in range(len(cat2)):
        if(r == 0):
            if(i == cat2[j][0]):
                sub.loc[sub['imfolder'] == i,'MGMT_value'] = np.float(cat2[j][1])**2
                r = r+1

In [ ]:
sub = sub.drop(['imfolder' , 't1' , 't2','t3' , 't4', 't5', 't6', 't7' , 't8'], axis = 1)
sub.to_csv('/kaggle/working/submission.csv', index = False)
sub